<a href="https://colab.research.google.com/github/ji-in/PyTorch/blob/main/autograd_tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%matplotlib inline


## A Gentle Introduction to ``torch.autograd``

``torch.autograd``는 자동으로 미분을 해주는 Pytorch의 엔진이다. 이번 섹션에서, 어떻게 autograd가 신경망 훈련을 도와주는지 살펴보자.

Background

신경망은 두가지 단계를 거쳐서 훈련된다.

1. Forward Propagation
Forward prop에서, 신경망은 정확한 출력을 위해 최선의 추측을 한다. 
최선의 추측을 위해 입력 데이터를 각 함수에 통과시킨다.

2. Backward Propagation
Backprop에서, 신경망은 추측의 에러(loss)를 사용하여 파라미터를 조정한다.
출력에서 시작해서 거꾸로 진행한다.
함수의 파라미터에 대해서 에러의 미분을 모으고 (gradients), 경사 하강법(gradient descent)을 사용해 파라미터를 최적화한다.
Backprop의 자세한 설명을 보고 싶다면, [3Blue1Brown](https://www.youtube.com/watch?v=tIeHLnjs5U8) 비디오를 확인해보자.


단순한 훈련 과정을 통해 `autograd`에 대해 배워보자.  
``torchvision``에서 pretrained resnet18 model을 불러온다.  
우리는 random data tensor를 사용해서 channel=3, height=64, width=64를 가진 이미지를 하나 만들고, 그 이미지에 상응하는 ``label``은 random values로 초기화했다. 



In [ ]:
import torch, torchvision
model = torchvision.models.resnet18(pretrained=True)
data = torch.rand(1, 3, 64, 64)
labels = torch.rand(1, 1000)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


예측을 하기 위해 입력 데이터를 모델의 각 레이어에 통과시킨다. **forward pass**




In [ ]:
prediction = model(data) # forward pass

모델의 에러(``loss``)를 계산하기 위해 모델의 예측값과 레이블을 사용한다.  
다음 단계는 네트워크를 통과시켜 에러를 backpropagate 하는 것이다.  
에러 tensor(``loss``)에서 ``.backward()``를 부르면 backward propagation이 시작된다.  
Autograd는 파라미터의 ``.grad`` 속성에 존재하는 각 모델 파라미터의 기울기를 계산하고 저장한다.

In [ ]:
loss = (prediction - labels).sum()
loss.backward() # backward pass

Optimizer를 부른다.  
이번 경우, optimizer=SGD, learning rate=0.01, momentum=0.9를 사용한다.
모델의 모든 파라미터를 optimizer에 등록한다.


In [ ]:
optim = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9)

마지막으로, ``.step()``를 불러 gradient descent를 초기화한다.  
Optimizer는 ``.grad``에 저장된 gradient에 따라 각 파라미터를 조정한다.



In [ ]:
optim.step() #gradient descent

신경망을 학습하기 위해 필요한 것은 전부 다 했다.  
아래의 섹션은 구체적인 autograd의 동작에 관한 것이다.  
건너 뛰어도 된다.


--------------




Differentiation in Autograd
~~~~~~~~~~~~~~~~~~~~~~~~~~~

``autograd``가 어떻게 gradients를 모으는지 확인해보자.  
``require_grad=True``를 사용해 두 개의 tensor인 ``a``와 ``b``를 만든다.
``require_grad=True``는 ``a``와 ``b``에 관련된 모든 연산을 추적하라고 ``autograd``에 신호를 보낸다.



In [ ]:
import torch

a = torch.tensor([2., 3.], requires_grad=True)
b = torch.tensor([6., 4.], requires_grad=True)

``a``와 ``b``를 사용해 또다른 tensor ``Q``를 만든다.

\begin{align}Q = 3a^3 - b^2\end{align}



In [ ]:
Q = 3*a**3 - b**2

``a``와 ``b``는 신경망의 파라미터, ``Q``는 에러라고 하자.    
신경망을 훈련할 때, 에러를 파라미터로 미분한 **에러의 gradients**가 필요하다. 즉,  

\begin{align}\frac{\partial Q}{\partial a} = 9a^2\end{align}

\begin{align}\frac{\partial Q}{\partial b} = -2b\end{align}


``Q``에서 ``.backward()``를 부르면, autograd는 gradients를 계산하고 ``.grad`` 속성을 가진 tensor에 gradients를 저장한다. ``gradient``가 vector이기 때문에 ``Q.backward()``에 ``gradient`` 를 명시적으로 전달해야한다.  
``Q``를 자기 자신에 대해 미분한 ``gradient``는 ``Q``와 shpae이 같은 tensor이다. 즉, 

\begin{align}\frac{dQ}{dQ} = 1\end{align}

``Q.sum().backward()``와 같이 Q를 scalar로 만들어서 backward를 부를 수도 있다.



In [ ]:
external_grad = torch.tensor([1., 1.])
Q.backward(gradient=external_grad)

Gradients는 ``a.grad``와 ``b.grad``에 저장된다.


In [ ]:
# gradient가 잘 구해졌는지 확인해보자.
print(9*a**2 == a.grad)
print(-2*b == b.grad)

tensor([True, True])
tensor([True, True])


**추가적으로 읽을 것 - ``autograd``를 사용해서 벡터 연산하기**


수학적으로, 벡터를 입력으로 받는 함수 $\vec{y}=f(\vec{x})$ 가 있다면, $\vec{y}$ 를 $\vec{x}$에 대해서 미분한 것은 Jacobian matrix $J$가 된다.:

\begin{align}J
     =
      \left(\begin{array}{cc}
      \frac{\partial \bf{y}}{\partial x_{1}} &
      ... &
      \frac{\partial \bf{y}}{\partial x_{n}}
      \end{array}\right)
     =
     \left(\begin{array}{ccc}
      \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{1}}{\partial x_{n}}\\
      \vdots & \ddots & \vdots\\
      \frac{\partial y_{m}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
      \end{array}\right)\end{align}

일반적으로, ``torch.autograd``는 vector와 Jacobian의 곱을 계산하는 엔진이다. 즉, 어떤 벡터 $v=(v _{1}  v _{2} ... v _{m} ) ^{T}$에 대해 $J^{T}\cdot \vec{v}$를 계산한다.

만약 $v$가 스칼라 함수 $l=g\left(\vec{y}\right)$의 기울기인 경우, $v=\left(\begin{array}{ccc}\frac{\partial l}{\partial y_{1}} & \cdots & \frac{\partial l}{\partial y_{m}}\end{array}\right)^{T}$ 이며,

Chain rule에 의해, vector와 Jacobian 곱은 $\vec{x}$에 대한 $l$의 미분이 된다.

\begin{align}J^{T}\cdot \vec{v}=\left(\begin{array}{ccc}
      \frac{\partial y_{1}}{\partial x_{1}} & \cdots & \frac{\partial y_{m}}{\partial x_{1}}\\
      \vdots & \ddots & \vdots\\
      \frac{\partial y_{1}}{\partial x_{n}} & \cdots & \frac{\partial y_{m}}{\partial x_{n}}
      \end{array}\right)\left(\begin{array}{c}
      \frac{\partial l}{\partial y_{1}}\\
      \vdots\\
      \frac{\partial l}{\partial y_{m}}
      \end{array}\right)=\left(\begin{array}{c}
      \frac{\partial l}{\partial x_{1}}\\
      \vdots\\
      \frac{\partial l}{\partial x_{n}}
      \end{array}\right)\end{align}

  
``external_grad``는 $\vec{v}$를 나타낸다.




Computational Graph

개념상으로, autograd는 텐서들과 모든 실행됐던 연산들을 directed acyclic graph (DAG)에 저장한다.
DAG에서, 리프 노드는 input tensors이고 루트 노드는 output tensors이다.
그래프의 루트에서부터 리프까지 탐색하면서, chain rule을 사용하여 자동적으로 gradients를 계산할 수 있다.

Forward pass에서, autograd는 동시에 두가지 일을 한다.

- 요청된 작업을 실행하여 결과 텐서를 계산한다.
- DAG에서 연산의 gradient function을 유지한다.

Backward pass는 DAG의 루트에서 `.backward()`가 불리면 시작된다. 그때 ``autograd``는

- 각 `.grad_fun`에서 gradients를 계산한다.
- gradients는 `.grad` 속성을 가진 텐서에 저장한다.
- chain rule을 사용해서, 리프 텐서까지 전파된다.

아래의 그림은 우리 예제를 가지고 DAG를 그린 것이다. 그래프에서, 화살표는 forward pass의 방향을 나타낸다. 노드들은 각 연산의 backward 함수를 나타낸다. 파랑색으로 나타낸 리프 노드는 우리의 리프 텐서인 `a`와 `b`를 나타낸다.

.. figure:: /_static/img/dag_autograd.png

<div class="alert alert-info"><h4>Note</h4><p>**DAGs are dynamic in PyTorch**
  주목해야 할 점은 그래프가 처음부터 다시 생성된다는 것이다.
각각의 ``.backward()``함수가 불리면, autograd는 새 그래프를 만들기 시작한다. 이것은 모델에서 control flow statements를 사용할 수 있도록 하는 것이다. 필요하다면 그래프의 모양, 크기, 연산을 매 iteration마다 바꿀 수 있다.</p></div>


Exclusion from the DAG


``torch.grad``는 ``requires_grad``가 ``True``로 설정된 모든 텐서의 연산을 추적한다. ``requires_grad``가 ``False``로 설정된 텐서들은 gradient computation DAG에서 배제된다.

한 개의 입력 텐서만 ``require_grad=True``일지라도, 연산의 결과 텐서는 gradients를 필요로 한다.



In [ ]:
x = torch.rand(5, 5)
y = torch.rand(5, 5)
z = torch.rand((5, 5), requires_grad=True)

a = x + y
print(f"Does `a` require gradients? : {a.requires_grad}")
b = x + z
print(f"Does `b` require gradients?: {b.requires_grad}")

Does `a` require gradients? : False
Does `b` require gradients?: True


신경망에서, gradients를 계산하지 않는 파라미터를 **frozen parameters**라고 한다.  
어떤 파라미터들의 gradients가 필요없는 것을 미리 안다면, "freeze"는 모델에서 유용하게 쓰인다. (autograd 계산을 줄여 모델의 성능을 높일 수 있다.)

DAG에서 제외되는 것이 중요한 것을 알려주는 또다른 경우는 [pretrained network를 finetuning 하는 경우이다.](https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html)

Finetuning에서, 모델의 대부분을 고정하고 새로운 레이블을 예측하기 위해 classifier 레이어만 수정한다.  
이것을 설명하기 위해 작은 예제를 살펴보자.  
위에서, 우리는 pretrained resnet18 모델을 부르고, 모든 파라미터들을 고정했다.


In [ ]:
from torch import nn, optim

model = torchvision.models.resnet18(pretrained=True)

# Freeze all the parameters in the network
for param in model.parameters():
    param.requires_grad = False

10개의 레이블을 가진 새로운 데이터 셋에서 모델을 finetune을 한다고 하자.  
Resnet에서, classifier는 마지막 레이어인 ``model.fc``이다.  
우리는 간단히 이 레이어를 우리의 classifier로 사용될 새로운 선형 레이어로 교체할 수 있다. (기본적으로 unfrozen 되었다.)



In [ ]:
model.fc = nn.Linear(512, 10)

``model.fc``의 파라미터를 제외한 모델의 모든 파라미터는 frozen되었다.  
Gradients를 계산할 파라미터는 ``model.fc``의 weights와 bias이다.


In [ ]:
# Optimize only the classifier
optimizer = optim.SGD(model.fc.parameters(), lr=1e-2, momentum=0.9)

모든 파라미터를 optimizer에 등록했지만, gradients를 계산하고 경사 하강법으로 업데이트 되는 파라미터들은 classifier의 weights와 bias이다.

동일한 제외 기능은 [torch.no_grad()](https://pytorch.org/docs/stable/generated/torch.no_grad.html)에서 이용 가능하다.

--------------




더 읽을만한 것들:


-  [In-place operations & Multithreaded Autograd](https://pytorch.org/docs/stable/notes/autograd.html)
-  [Example implementation of reverse-mode autodiff](https://colab.research.google.com/drive/1VpeE6UvEPRz9HmsHh1KS0XxXjYu533EC)

